## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# Business Understanding



In [1]:
import pandas as pd
df = pd.read_csv('Data/data-clean.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
display(df.head(2)),(display(df.info()))

,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,genres,director
0,Pirates of the Caribbean: On Stranger Tides 2011,410600000,241063875,1045663875,804600000,635063875,2011,"['Action', 'Adventure', 'Fantasy']",['Rob Marshall']
1,Dark Phoenix 2019,350000000,42762350,149762350,107000000,-200237650,2019,"['Action', 'Adventure', 'Sci-Fi']",['Simon Kinberg']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1349 entries, 0 to 1348
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie              1349 non-null   object
 1   production_budget  1349 non-null   int64 
 2   domestic_gross     1349 non-null   int64 
 3   worldwide_gross    1349 non-null   int64 
 4   foreign_gross      1349 non-null   int64 
 5   profit             1349 non-null   int64 
 6   year               1349 non-null   int64 
 7   genres             1349 non-null   object
 8   director           1349 non-null   object
dtypes: int64(6), object(3)
memory usage: 95.0+ KB


None

(None, None)